# INFO

- ĐỒ ÁN CUỐI KÌ MÔN `LẬP TRÌNH CHO KHOA HỌC DỮ LIỆU`
- Thành viên:
    + 18120517 Nguyễn Công Bình Phương
    + 18120514 Phan Thiên Phúc

# OUR ANALYSIS

In [8]:
#import additional packages to env
!conda install -c conda-forge seaborn -y

Solving environment: done

# All requested packages already installed.



In [17]:
#import library
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### ABOUT DATA

nguồn: https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016

- Content: This compiled dataset pulled from four other datasets linked by time and place, and was built to find signals correlated to increased suicide rates among different cohorts globally, across the socio-economic spectrum.

- References: 
    United Nations Development Program. (2018). Human development index (HDI). Retrieved from http://hdr.undp.org/en/indicators/137506

    World Bank. (2018). World development indicators: GDP (current US$) by country:1985 to 2016. Retrieved from http://databank.worldbank.org/data/source/world-development-indicators#

    [Szamil]. (2017). Suicide in the Twenty-First Century [dataset]. Retrieved from https://www.kaggle.com/szamil/suicide-in-the-twenty-first-century/notebook

    World Health Organization. (2018). Suicide prevention. Retrieved from http://www.who.int/mental_health/suicide-prevention/en/

- Inspiration: Suicide Prevention.


***Data dictionary***:
- Country: Column representing countries (string)
- Year: Column representing years (integer)
- Sex: Column representing genres (string)
- Age: Column representing ages (string)
- Suicides_no: Column representing the number of suicides (integer)
- Population: Column representing the population (integer)
- Suicides / 100k pop: Column that represents the number of suicides per 100 thousand inhabitants (float)
- Country-year: Column that represents (string)
- HDI for year: Column that represents the HDI in each year (float)
- Gdp_for_year($): Column representing GDP in each year (string)
- Gdp_per_capita($): Column representing GDP per capita (float)
- Generation: Column that represents the generation (string)

In [10]:
df = pd.read_csv('master.csv')
df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   country             27820 non-null  object 
 1   year                27820 non-null  int64  
 2   sex                 27820 non-null  object 
 3   age                 27820 non-null  object 
 4   suicides_no         27820 non-null  int64  
 5   population          27820 non-null  int64  
 6   suicides/100k pop   27820 non-null  float64
 7   country-year        27820 non-null  object 
 8   HDI for year        8364 non-null   float64
 9    gdp_for_year ($)   27820 non-null  object 
 10  gdp_per_capita ($)  27820 non-null  int64  
 11  generation          27820 non-null  object 
dtypes: float64(2), int64(4), object(6)
memory usage: 2.5+ MB


### DATA ANALYSIS

In [15]:
df.shape

(27820, 12)

In [23]:
df.duplicated().any()

False

In [24]:
df.isnull().values.any()

True

In [27]:
def check_col_contain_null(s):
    return s.isnull().values.any()

df.agg(check_col_contain_null)

country               False
year                  False
sex                   False
age                   False
suicides_no           False
population            False
suicides/100k pop     False
country-year          False
HDI for year           True
 gdp_for_year ($)     False
gdp_per_capita ($)    False
generation            False
dtype: bool

### INSIGHT 2